In [23]:
import ee

In [24]:
ee.Authenticate()

True

In [25]:
ee.Initialize(project='vegetation--sav')

In [42]:
dem = ee.Image('USGS/3DEP/10m')
dem.getInfo()

{'type': 'Image',
 'bands': [{'id': 'elevation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [3650412, 939612],
   'crs': 'EPSG:4269',
   'crs_transform': [9.259259259299957e-05,
    0,
    -174.0005555570324,
    0,
    -9.259259259299957e-05,
    72.00055555584566]}],
 'version': 1648043371709135,
 'id': 'USGS/3DEP/10m',
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[8.533243855035293, 72.00060200570647],
    [-36.52255593943564, 72.00060196804901],
    [-106.00421059777325, 72.00060205939565],
    [-174.0008093445371, 72.00060192652217],
    [-174.0006053017384, -15.000601990025386],
    [-134.22597532324247, -15.000602004951233],
    [-106.00421062932261, -15.000602024344811],
    [-63.754083308670964, -15.00060201818635],
    [-26.290103199499303, -15.000601982388343],
    [34.939573443906426, -15.000602040237249],
    [91.87822164793576, -15.000602000123632],
    [132.47795335015223, -15.000602038567253],
    [164.

In [27]:
geom = ee.Geometry.Point([-91.0989573,30.3529013])
geom_col = ee.FeatureCollection([geom])
geom_col

In [28]:
elev = dem.sampleRegions(geom_col).getInfo()
elev

{'type': 'FeatureCollection',
 'columns': {},
 'properties': {'band_order': ['elevation']},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '0_0',
   'properties': {'elevation': 3.220296621322632}}]}

In [29]:
#read from the csv file
import pandas as pd
table = pd.read_csv(r"C:\Users\hpokha1\OneDrive - Louisiana State University\Documents\GitHub\geog4057\project\data\boundary.csv")

In [30]:
table

,col,row,X,Y
0,4871,174,699102.887792,186780.445813
1,4871,174,699102.887792,186780.445813
2,4872,174,699105.887419,186780.445813
3,4870,175,699099.888166,186777.446186
4,4873,174,699108.887046,186780.445813
...,...,...,...,...
333,4825,232,698964.904966,186606.467466
334,4826,234,698967.904593,186600.468213
335,4825,233,698964.904966,186603.467840
336,4825,234,698964.904966,186600.468213


In [31]:
ra1 = arcpy.Raster(r"C:\Users\hpokha1\OneDrive - Louisiana State University\Documents\GitHub\geog4057\project\data\flood_2class.tif")
ra1.spatialReference.factoryCode

32119

In [32]:
import geopandas
gdf = geopandas.GeoDataFrame(table)
gdf.set_geometry(
    geopandas.points_from_xy(gdf['X'], gdf['Y']),
    inplace=True, crs=f'EPSG:{ra1.spatialReference.factoryCode}')

In [33]:
gdf.to_file(r'C:\Users\hpokha1\OneDrive - Louisiana State University\Documents\GitHub\geog4057\project\data\boundary.shp')

In [34]:
shapefile =r'C:\Users\hpokha1\OneDrive - Louisiana State University\Documents\GitHub\geog4057\project\data\boundary.shp'

In [35]:
#add field to the shapefile
arcpy.management.AddField(shapefile,'elevation',"FLOAT")

<Result 'C:\\Users\\hpokha1\\OneDrive - Louisiana State University\\Documents\\GitHub\\geog4057\\project\\data\\boundary.shp'>

In [41]:
#read elevation fro gee and write values to the shapefile
with arcpy.da.UpdateCursor(shapefile,
                           ['SHAPE@XY','elevation'],
                           spatial_reference=4326) as cursor:
    for row in cursor:
        X,Y =row[0]
        geom = ee.Geometry.Point([X,Y])
        geom_col = ee.FeatureCollection([geom])
        elev = dem.sampleRegions(geom_col)
        elevation = elev.getInfo()['features'][0]['properties']['elevation']
        print(elevation)
        row[1]=elevation 
        cursor.updateRow(row)

22.477031707763672
22.477031707763672
22.477031707763672
22.24553871154785
22.477031707763672
22.24553871154785
22.682371139526367
22.403953552246094
22.403953552246094
22.682371139526367
22.682371139526367
22.682371139526367
22.176103591918945
22.682371139526367
22.176103591918945
22.242366790771484
22.343332290649414
22.343332290649414
22.242366790771484
22.242366790771484
22.343332290649414
22.242366790771484
22.343332290649414
22.343332290649414
22.32790184020996
22.32790184020996
22.838306427001953
22.32790184020996
22.838306427001953
22.32790184020996
22.838306427001953
22.838306427001953
22.368783950805664
22.751964569091797
22.751964569091797
22.368783950805664
22.368783950805664
22.751964569091797
22.368783950805664
22.368783950805664
22.751964569091797
22.1630802154541
22.1630802154541
22.668912887573242
22.668912887573242
22.668912887573242
22.44307518005371
22.668912887573242
22.296045303344727
22.44307518005371
22.284040451049805
22.44307518005371
22.284040451049805
22.706